imports

In [12]:
import pandas as pd

writer = pd.ExcelWriter('./results/data_of_all.xlsx', engine='xlsxwriter')

config

In [13]:
config_prefix = ['BR', 'BI', 'TR', 'TI']

config_datasheet = r'./results/preprocessed.xlsx'

import df

In [14]:
raw = pd.read_excel(config_datasheet)
df = pd.DataFrame(raw)
df["Flag"] = df["Flag"].apply(str)

remove outliers

In [15]:
#split dataframes into dataframes for the 2*2 related factors
df_BU_R = df.loc[(df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="BU")].drop("ClickData", axis=1)
df_BU_I = df.loc[(df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="BU")].drop("ClickData", axis=1)
df_TD_R = df.loc[(df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="TD")].drop("ClickData", axis=1)
df_TD_I = df.loc[(df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="TD")].drop("ClickData", axis=1)

In [16]:
df_array_independent = [df_BU_R, df_BU_I, df_TD_R, df_TD_I]
df_means = [float(df["ResponseTime"].mean()) for df in df_array_independent]
df_stds = [float(df["ResponseTime"].std()) for df in df_array_independent]

for idx, df in enumerate(df_array_independent):
    mean = float(df_means[idx])
    stddev = float(df_stds[idx])
    for row_idx, row in df.iterrows():
        response_time = row["ResponseTime"]
        if mean+1.5*stddev < response_time < mean+2*stddev:
            df.loc[row_idx, "Flag"] = "notNormal"
            
        if response_time <= 10 and row["Correctness"] == False or response_time >= mean+2*stddev:
            df.loc[row_idx, "Flag"] = "Outlier"

df = df_BU_R
df = df.append(df_BU_I)
df = df.append(df_TD_R)
df = df.append(df_TD_I)
df.to_excel(writer, "labeled")

print(len(df))

for idx, df in enumerate(df_array_independent):
    df.loc[df["Flag"]=="Outlier", "ResponseTime"] = df_means[idx]

650


In [19]:
for df_idx, _df in enumerate(df_array_independent):
    for row_idx, row in df.iterrows():
        id_value = row["Subject"]
        entries_id_BU_R = df_BU_R.loc[(df_BU_R["Subject"] == id_value) & (df_BU_R["Flag"] == "Outlier")]
        entries_id_BU_I = df_BU_I.loc[(df_BU_I["Subject"] == id_value) & (df_BU_I["Flag"] == "Outlier")] 
        entries_id_TD_R = df_TD_R.loc[(df_TD_R["Subject"] == id_value) & (df_TD_R["Flag"] == "Outlier")]
        entries_id_TD_I = df_TD_I.loc[(df_TD_I["Subject"] == id_value) & (df_TD_I["Flag"] == "Outlier")]
        
        df_per_id = entries_id_BU_R
        df_per_id = df_per_id.append(entries_id_BU_I)
        df_per_id = df_per_id.append(entries_id_TD_R)
        df_per_id = df_per_id.append(entries_id_TD_I)
        
        number_of_outliers_per_subject = len(df_per_id)
        
        if number_of_outliers_per_subject == 2:
            
            df_nn_BU_R = df_BU_R.loc[(df_BU_R["Subject"] == id_value) & (df_BU_R["Flag"] == "notNormal")]
            df_nn_BU_I = df_BU_I.loc[(df_BU_I["Subject"] == id_value) & (df_BU_I["Flag"] == "notNormal")]
            df_nn_TD_R = df_TD_R.loc[(df_TD_R["Subject"] == id_value) & (df_TD_R["Flag"] == "notNormal")]
            df_nn_TD_I = df_TD_I.loc[(df_TD_I["Subject"] == id_value) & (df_TD_I["Flag"] == "notNormal")]
            
            num_non_normals = len(df_nn_BU_R) + len(df_nn_BU_I) + len(df_nn_TD_R) + len(df_nn_TD_I)
            
            if num_non_normals > 0:
                print("Participant with the following number got deleted: " + str(id_value) + ". Because 2 where missing and " + str(num_non_normals) + " datapoints where not normal.")
                df_BU_R = df_BU_R.loc[df_BU_R["Subject"]!=id_value]
                df_BU_I = df_BU_I.loc[df_BU_I["Subject"]!=id_value]
                df_TD_R = df_TD_R.loc[df_TD_R["Subject"]!=id_value]
                df_TD_I = df_TD_I.loc[df_TD_I["Subject"]!=id_value]
                
        if number_of_outliers_per_subject > 2:
            print("Participant with the following number got deleted: " + str(id_value) + ". Because " + str(number_of_outliers_per_subject) + " where missing.")
            df_BU_R = df_BU_R.loc[df_BU_R["Subject"]!=id_value]
            df_BU_I = df_BU_I.loc[df_BU_I["Subject"]!=id_value]
            df_TD_R = df_TD_R.loc[df_TD_R["Subject"]!=id_value]
            df_TD_I = df_TD_I.loc[df_TD_I["Subject"]!=id_value]

df_array_independent = [df_BU_R, df_BU_I, df_TD_R, df_TD_I]
print(sum([len(df) for df in df_array_independent]))

151
618


In [20]:
algo_name_array = [name for name in df["Algorithm"].unique()]

split dataframes 

In [88]:
df_cat_array = df_array_independent

df = df_array_independent[0]
df = df.append(df_array_independent[1])
df = df.append(df_array_independent[2])
df = df.append(df_array_independent[3])

df_algo_cat_array = []
for algo_name in algo_name_array:
    tmp_df = []
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="BU")])
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="BU")])
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="TD")])
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="TD")])
    df_algo_cat_array.append(tmp_df)

create result

In [89]:
cols = ["Algorithm", "Comprehension",
        "rec correct", "rec total Answers", "rec percent", "R_ResponseTimeMean", "R_ResponseTimeDeviation",
        "it correct", "it total Answers", "it percent", "I_ResponseTimeMean", "I_ResponseTimeDeviation"]

result_df = pd.DataFrame(columns=cols)

count_cols = ["Algorithm", "Comprehension", "Programming", "Right", "Total"]
count_df = pd.DataFrame(columns=count_cols)

for algo_df_array in df_algo_cat_array:
    for i in range(2):
        rec =  algo_df_array[i*2+0]
        it =  algo_df_array[i*2+1]
        
        Algorithm = rec.iloc[0]["Algorithm"]
        Comprehension = "Bottom-Up"
        if i == 1:
            Comprehension = "Top-Down"
            
        count_df = count_df.append(pd.DataFrame([[Algorithm, Comprehension, "R", len(rec.loc[rec["Correctness"]==True]) ,len(rec)]], columns=count_cols))
        count_df = count_df.append(pd.DataFrame([[Algorithm, Comprehension, "I", len(it.loc[it["Correctness"]==True]) ,len(it)]], columns=count_cols))
        
        R_ResponseTimeMean = int(rec["ResponseTime"].mean())
        R_ResponseTimeDeviation = int(rec["ResponseTime"].std())
        
        I_ResponseTimeMean = int(it["ResponseTime"].mean())
        I_ResponseTimeDeviation = int(it["ResponseTime"].std())
        
        rec_cor = len(rec.loc[rec["Correctness"]==True])
        rec_total = len(rec)
        rec_percent = rec_cor/rec_total
        
        it_cor = len(it.loc[it["Correctness"]==True])
        it_total = len(it)
        it_percent = it_cor/it_total
        
        data = [
            Algorithm, Comprehension,
            rec_cor, rec_total, rec_percent, R_ResponseTimeMean, R_ResponseTimeDeviation,
            it_cor, it_total, it_percent, I_ResponseTimeMean, I_ResponseTimeDeviation,
        ]
        
        result_df = result_df.append(pd.DataFrame([data], columns=cols))
        



for i in range(2):
    rec =  df_cat_array[i*2+0]
    it =  df_cat_array[i*2+1]
    
    Algorithm = "Total"
    Comprehension = "Top-Down"
    if i == 1:
        Comprehension = "Bottom-Up"
    
    R_ResponseTimeMean = int(rec["ResponseTime"].mean())
    R_ResponseTimeDeviation = int(rec["ResponseTime"].std())
    
    I_ResponseTimeMean = int(it["ResponseTime"].mean())
    I_ResponseTimeDeviation = int(it["ResponseTime"].std())
    
    rec_cor = len(rec.loc[rec["Correctness"]==True])
    rec_total = len(rec)
    rec_percent = rec_cor/rec_total
    
    it_cor = len(it.loc[it["Correctness"]==True])
    it_total = len(it)
    it_percent = it_cor/it_total
    
    data = [
        Algorithm, Comprehension,
        rec_cor, rec_total, rec_percent, R_ResponseTimeMean, R_ResponseTimeDeviation,
        it_cor, it_total, it_percent, I_ResponseTimeMean, I_ResponseTimeDeviation,
    ]
    
    result_df = result_df.append(pd.DataFrame([data], columns=cols))
    
df_recursive = df_array_independent[0]     
df_recursive = df_recursive.append(df_array_independent[2])

df_iterative = df_array_independent[1]     
df_iterative = df_iterative.append(df_array_independent[3])

rec_cor = len(df_recursive.loc[df_recursive["Correctness"]==True])
rec_total = len(df_recursive)
rec_percent = rec_cor/rec_total

it_cor = len(df_iterative.loc[df_iterative["Correctness"]==True])
it_total = len(df_iterative)
it_percent = it_cor/it_total

result_df = result_df.append(pd.DataFrame(
    [["Total", "Total",
    rec_cor, rec_total, rec_percent, int(df_recursive["ResponseTime"].mean()), int(df_recursive["ResponseTime"].std()),
    it_cor, it_total, it_percent, int(df_iterative["ResponseTime"].mean()), int(df_iterative["ResponseTime"].std())]],
    columns=cols))

print("Write total means to excel sheet")
result_df.to_excel(writer, sheet_name="corrects.info")
writer.save()
result_df

Write total means to excel sheet


,Algorithm,Comprehension,rec correct,rec total Answers,rec percent,R_ResponseTimeMean,R_ResponseTimeDeviation,it correct,it total Answers,it percent,I_ResponseTimeMean,I_ResponseTimeDeviation
0,BinarySearch,Bottom-Up,6,7,0.857143,368,157,2,13,0.153846,187,145
0,BinarySearch,Top-Down,3,13,0.230769,226,126,9,16,0.562500,211,158
0,BubbleSort,Bottom-Up,13,25,0.520000,232,84,8,25,0.320000,204,169
0,BubbleSort,Top-Down,17,28,0.607143,36,39,15,29,0.517241,222,102
0,Factorial,Bottom-Up,24,27,0.888889,93,66,26,29,0.896552,127,81
0,Factorial,Top-Down,24,25,0.960000,78,36,18,26,0.692308,112,101
0,Fibonacci,Bottom-Up,12,25,0.480000,174,136,17,24,0.708333,212,132
0,Fibonacci,Top-Down,15,28,0.535714,161,79,23,27,0.851852,182,111
0,IntegerBinary,Bottom-Up,6,14,0.428571,167,113,6,8,0.750000,224,117
0,IntegerBinary,Top-Down,11,14,0.785714,157,101,12,13,0.923077,207,94
